In [46]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
from env import github_token, github_username
import re
import numpy as np
import json
from typing import Dict, List, Optional, Union, cast
import requests
import pandas as pd

In [47]:
"""
A module for obtaining repo readme and language data from the github API.

Before using this module, read through it, and follow the instructions marked
TODO.

After doing so, run it like this:

    python acquire.py

To create the `data.json` file that contains the data.
"""

# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token: https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.


headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data(REPOS) -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


#if __name__ == "__main__":
#    data = scrape_github_data()
#    json.dump(data, open("data.json", "w"), indent=1)

def get_dataframe(REPOS):
    """
    This function takes a list of github repos, cache the data and returns the url, programming language and readme.
    """
    file = 'data.json'
    
    if os.path.exists(file):
        
        return pd.read_json('data.json')
    else:
        data = scrape_github_data(REPOS)
        json.dump(data, open("data.json", "w"), indent=1)
        return pd.read_json('data.json')

In [3]:
df = pd.read_csv('links.csv').drop(columns = 'Unnamed: 0')
df['href'] = df['href'].str[1:]
df

,href
0,learn-co-students/simple-blackjack-cli-prework
1,rocketacademy/basics-blackjack
2,datamllab/rlcard
3,learn-co-students/simple-blackjack-cli-001-pre...
4,cocos-creator/tutorial-blackjack-deprecated
...,...
435,TigerZen/Unity3D-BlackJack-Dapp
436,merdoyovski/Modified-Blackjack_Reach-Bounty-Hack
437,DEEBAZZ890/Blackjack-game
438,cs361-W16/Blackjack-14


In [6]:
df2 = get_dataframe(df.href)
df2

,repo,language,readme_contents
0,learn-co-students/simple-blackjack-cli-prework,Ruby,# Simplified Blackjack - Procedural\n\n## Lear...
1,rocketacademy/basics-blackjack,HTML,# Rocket Academy Coding Basics: Blackjack\n
2,datamllab/rlcard,Python,# RLCard: A Toolkit for Reinforcement Learning...
3,learn-co-students/simple-blackjack-cli-001-pre...,Ruby,# Simplified Blackjack - Procedural\n\n## Lear...
4,cocos-creator/tutorial-blackjack-deprecated,JavaScript,# 21点游戏 - Cocos Creator 制造\n\n「21点游戏」是 Cocos C...
...,...,...,...
435,TigerZen/Unity3D-BlackJack-Dapp,C#,# Unity3D-BlackJack-Dapp\n\n教你使用Unity3D製作Dapp ...
436,merdoyovski/Modified-Blackjack_Reach-Bounty-Hack,Shell,# Reach Modified Blackjack DApp\n\nThis is a m...
437,DEEBAZZ890/Blackjack-game,Python,# 21/Blackjack-game\n\nFrom a single deck of c...
438,cs361-W16/Blackjack-14,Java,[![Build Status](https://travis-ci.org/cs361-W...


In [7]:
df2.to_csv('original_data.csv', index = False)

In [9]:
df2.language.unique()

array(['Ruby', 'HTML', 'Python', 'JavaScript', None, 'Kotlin', 'Java',
       'Lua', 'CSS', 'Vue', 'Rust', 'Swift', 'C#', 'Objective-C', 'C++',
       'Jupyter Notebook', 'TypeScript', 'Elixir', 'Solidity', 'PHP',
       'Go', 'Clojure', 'Dart', 'Roff', 'C', 'Visual Basic', 'Haskell',
       'Shell'], dtype=object)

In [15]:
df2.language[df2.language == 'Jupyter Notebook'] = 'Python'

In [17]:
df2.language.unique()

array(['Ruby', 'HTML', 'Python', 'JavaScript', None, 'Kotlin', 'Java',
       'Lua', 'CSS', 'Vue', 'Rust', 'Swift', 'C#', 'Objective-C', 'C++',
       'TypeScript', 'Elixir', 'Solidity', 'PHP', 'Go', 'Clojure', 'Dart',
       'Roff', 'C', 'Visual Basic', 'Haskell', 'Shell'], dtype=object)

In [18]:
df2.language.value_counts()

Ruby            216
Java             52
JavaScript       40
Python           39
C#               15
C++               9
Swift             7
HTML              5
CSS               5
TypeScript        4
Lua               3
Objective-C       3
Vue               2
Rust              2
Kotlin            2
PHP               2
Go                2
Dart              2
C                 1
Haskell           1
Visual Basic      1
Solidity          1
Roff              1
Clojure           1
Elixir            1
Shell             1
Name: language, dtype: int64

In [19]:
df2.language[df2.language == 'Objective-C'] = 'C_based'

In [20]:
df2.language[df2.language == 'C#'] = 'C_based'

In [21]:
df2.language[df2.language == 'C++'] = 'C_based'

In [22]:
df2.language[df2.language == 'C'] = 'C_based'

In [23]:
df2.language.value_counts()

Ruby            216
Java             52
JavaScript       40
Python           39
C_based          28
Swift             7
CSS               5
HTML              5
TypeScript        4
Lua               3
Kotlin            2
Vue               2
Rust              2
PHP               2
Go                2
Dart              2
Roff              1
Haskell           1
Visual Basic      1
Solidity          1
Clojure           1
Elixir            1
Shell             1
Name: language, dtype: int64

In [29]:
df2.language[(df2.language == 'Swift') 
            | (df2.language == 'CSS') 
            | (df2.language == 'HTML')
            | (df2.language == 'TypeScript')
            | (df2.language == 'Lua')
            | (df2.language == 'Kotlin')
            | (df2.language == 'Vue')
            | (df2.language == 'Rust')
            | (df2.language == 'PHP')
            | (df2.language == 'Go')
            | (df2.language == 'Dart')
            | (df2.language == 'Roff')
            | (df2.language == 'Haskell')
            | (df2.language == 'Visual Basic')
            | (df2.language == 'Solidity')
            | (df2.language == 'Clojure')
            | (df2.language == 'Elixir')
            | (df2.language == 'Shell')] = 'Other'

In [30]:
df2.language.value_counts()

Ruby          216
Java           52
Other          43
JavaScript     40
Python         39
C_based        28
Name: language, dtype: int64

In [42]:
df2 = df2[df2.readme_contents != '']

In [43]:
df2.language.value_counts()

Ruby          213
Other          38
Python         36
Java           36
JavaScript     35
C_based        22
Name: language, dtype: int64

In [45]:
df2.to_csv('exploration_data.csv', index = False)